#Training
Todos nuestros archivos propios (TFrecord, pbtxt) se subieron a google drive, por lo que configuramos colab para tener acceso al drive.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Se instaló la API de Tensorflow que incluía a *model zoo*, entre otras herramientas. Tambíen se instalaron otros paquetes por medio de pip como *dependencies*.

In [0]:
!git clone https://github.com/tensorflow/models.git
import sys
sys.path.append('models/research')
sys.path.append('models/research/object_detection')

In [6]:
!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

Selecting previously unselected package libprotobuf10:amd64.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../libprotobuf10_3.0.0-9ubuntu5_amd64.deb ...
Unpacking libprotobuf10:amd64 (3.0.0-9ubuntu5) ...
Selecting previously unselected package libprotoc10:amd64.
Preparing to unpack .../libprotoc10_3.0.0-9ubuntu5_amd64.deb ...
Unpacking libprotoc10:amd64 (3.0.0-9ubuntu5) ...
Selecting previously unselected package protobuf-compiler.
Preparing to unpack .../protobuf-compiler_3.0.0-9ubuntu5_amd64.deb ...
Unpacking protobuf-compiler (3.0.0-9ubuntu5) ...
Setting up libprotobuf10:amd64 (3.0.0-9ubuntu5) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up libprotoc10:amd64 (3.0.0-9ubuntu5) ...
Setting up protobuf-compiler (3.0.0-9ubuntu5) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
/content/models/research
.....................
----------------------------------------------------------------------
Ran 21 tests in 0

A continuación se descarga el modelo preentrenado para el modelo que elegimos, faster_rcnn_inception_v2_coco

In [8]:
%cd /content
import os
import shutil
import glob
import urllib
import tarfile

MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'
if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content


Editamos el archivo .config con nuestros parámetros. Se cambia el trainining record, el validation record, el label map, el modelo inicial (este viene siendo el descargado en el paso anterior) y el número de clases.

In [0]:
import re

filename = '/content/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config'
with open(filename) as f:
  s = f.read()
with open(filename, 'w') as f:
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/content/pretrained_model/model.ckpt', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record-\?\?\?\?\?-of-00100', '/content/gdrive/My Drive/train.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record-\?\?\?\?\?-of-00010', '/content/gdrive/My Drive/test.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', '/content/gdrive/My Drive/label_map.pbtxt', s)
  s = re.sub('num_classes: 90', 'num_classes: 13', s)
  f.write(s)

Se corre el script que entrena la red.

In [18]:
!python models/research/object_detection/model_main.py \
    --pipeline_config_path=models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config \
    --model_dir=trained \
    --alsologtostderr \
    --num_train_steps=3000 \
    --num_eval_steps=500

/content/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "models/research/object_detection/model_main.py", line 26, in <module>
    from object_detection import model_lib
  File "/content/models/research/object_detection/model_lib.py", line 27, in <module>
    from object_detection import eval_util
  File "/content/models/research/object_detection/eval_util.py", line 27, in <module>
    from object_detection.metrics import coco_evaluation
  File "/content/models/research/object_detection/metrics/coco_evaluation.py", line 20, in <module>
    from object_detection.metrics import coco_tools
  File "/content/models/research/object_detection/met

Se guarda el último checkpoint guardado para a partir de ahí general el frozen model que se le dará a nuestra aplicación.

In [20]:
import os
lst = os.listdir('trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config \
    --output_directory='/content/gdrive/My Drive/fine_tuned_model' \
    --trained_checkpoint_prefix=trained/$last_model

Instructions for updating:
keep_dims is deprecated, use keepdims instead
W0928 00:23:53.495687 139948296075136 tf_logging.py:125] From /content/models/research/object_detection/predictors/heads/box_head.py:93: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0928 00:23:54.543430 139948296075136 tf_logging.py:125] From /content/models/research/object_detection/exporter.py:280: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for deta

Las siguientes herramientas sirven para poder usar Tensorboard.

In [15]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


Redirecting output to ‘wget-log’.
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


A esto le damos el directorio donde se entrenó el modelo y al correr la celda nos arrojará una liga donde podemos ver el Tensorboard del entrenamiento.

In [16]:
LOG_DIR = 'trained'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://babaf81b.ngrok.io
